Pydantic validation of LLM response to make sure its in json format with right data types.

In [1]:
!pip install --upgrade fireworks-ai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 577.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install pydantic==2.5.0
!pip install pydantic-core==2.14.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.3 MB/s eta 0:00:00


In [ ]:

import fireworks.client



In [ ]:
fireworks.client.api_key = "W4etuSqWlXPUIa8gsuz8g9kCvz6KGNsSF4q8FePNjpBLOgfF"

Web scrapping https://en.wikipedia.org/wiki/List_of_2023_box_office_number-one_films_in_the_United_States .
I have only extracted data from the table



In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_box_office(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'wikitable'})
        box_office_data = []
        for row in table.find_all('tr')[1:]:
            columns = row.find_all(['th', 'td'])
            data = [column.get_text(strip=True) for column in columns]
            box_office_data.append(data)

        return box_office_data
    else:
        print(f"Error: Unable to fetch data. Status Code: {response.status_code}")

url = 'https://en.wikipedia.org/wiki/List_of_2023_box_office_number-one_films_in_the_United_States'

result = scrape_box_office(url)
string = ""
for row in result:
    for i in range(len(row)):
      if i==0:
        pass
      else:
        string += row[i] + " "
    string += '\n'


print(string)

Specifying a pydantic schema

In [ ]:
from pydantic import BaseModel, ValidationError
import json
class WebScrape1(BaseModel):
    date: str
    film: str
    gross: int

In [ ]:
response_schema_dict = WebScrape1.model_json_schema()
response_schema_json = json.dumps(response_schema_dict)
response_schema_json

Creating a prompt

In [ ]:
# question = "1 January 8, 2023 Avatar: The Way of Water $45,838,986 Black Panther: Wakanda ForeverandAvatar: The Way of Waterbecame the first two films to consecutively top the box office for four consecutive weekends each sinceThe Hunger Games: Mockingjay – Part 2andStar Wars: The Force Awakensin 2015 and 2016. [2] "
prompt = f"""I will provide you with a unstructured data and you have to convert it into the following format and Do not include the provided properties json in the output:
```json
{response_schema_dict}
```
Make sure you only have the required json as output and no other text.

Data: {string}

only generate extracted data json.
The provided format should be strictly followed.

"""

In [ ]:
prompt

Creating functions for text generation and error handling

In [ ]:
def askLLM(prompt):
    completion = fireworks.client.ChatCompletion.create(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    messages=[
      {
        "role": "user",
        "content": prompt,
      }
    ],
    n=1,
    max_tokens=4000,
    temperature=0.1,
    top_p=0.9,
   )
    return completion.choices[0].message.content


def handle_validation_failure(error, prompt, num_tries):
  if num_tries > 3:
    return "Couldnt validate LLM response"
  else:
    out = askLLM(error + prompt+ "Make sure the output is json")
    print(out)
    return out


Providing initial prompt and viewing the results

In [ ]:
completion = askLLM(prompt)

In [ ]:
completion

In [ ]:
max_tries = 0

In [ ]:
while(max_tries < 3):
  try:
    json_objects = json.loads(completion)
    break
  except:
    completion = handle_validation_failure("Output Not json", prompt, max_tries)
    max_tries+=1


Pydantic Validations for each individual json extraction. This approach is used in order to avoid regeneration everytime there is an error in a single json. Errors are extracted and displayed to the user

In [ ]:
failed = []
validated = []
for i in range(len(json_objects)):
  try:
      validated_data = WebScrape1.parse_obj(json_objects[i])
      print("Validation successful!")
      validated.append(json_objects[i])
      print("Validated data:", validated_data.dict())
  except ValidationError as e:
      print("Validation failed!")
      failed.append(json_objects[i])
      print(e.json())

print(failed)

Displaying the failed prompts

In [ ]:
for i in failed:
  print(i)

In [ ]:
for i in validated:
  print(i)

Sorting by date

In [ ]:
prompt = f"""I will provide you with a unstructured data and you have to convert it into the following format and Do not include the provided properties json in the output:
```json
{response_schema_dict}
```
Make sure you only have the required json as output and no other text.

Data: {string}

only generate extracted data json. Sort the data according to the gross revenue.
The provided format should be strictly followed.

"""

In [ ]:
completion_new = askLLM(prompt)

In [ ]:
max_tries = 0

In [ ]:

while(max_tries < 3):
  try:
    json_objects1 = json.loads(completion_new)
    break
  except:
    completion_new = handle_validation_failure("Output Not json, do not produce schema in the response again", prompt, max_tries)
    max_tries+=1
    print(f"Try {max_tries}")

In [ ]:
completion_new

In [ ]:
json_objects1

In [ ]:
failed = []
validated = []
for i in range(len(json_objects1)):
  try:
      validated_data = WebScrape1.parse_obj(json_objects1[i])
      print("Validation successful!")
      validated.append(json_objects1[i])
      print("Validated data:", validated_data.dict())
  except ValidationError as e:
      print("Validation failed!")
      failed.append(json_objects1[i])
      print(e.json())

print(failed)

In [ ]:
for i in validated:
  print(i)